[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/dakimura/jquants-sample/blob/main/data_retrieve.ipynb)

# J-Quants APIを用いた分析用データ取得

本ノートブックでは、データ分析に先立って
分析で必要となる株価・財務情報データをJ-Quants APIを使用して取得し、
保存するところまでを行います。
Google Colab上で動作確認を行っています。


## J-Quants API
J-Quants APIを利用するためには [J-Quants API の Web サイト](https://application.jpx-jquants.com/) から取得できるリフレッシュトークンが必要になります。
会員登録とリフレッシュトークンの取得を完了させてから次に進みます。

必要なライブラリのinstall/importと設定を行います。

In [16]:
!pip install numpy pandas requests jquants-api-client==0.1.2

import getpass
from requests import HTTPError
from datetime import datetime

import jquantsapi
import pandas as pd
import numpy as np
from dateutil import tz

# --- コンフィグ ---
# Googleドライブをマウントするディレクトリ
GOOGLE_DRIVE_MOUNT_DIR_PATH = "/content/drive/"

# データを保存するGoogleドライブ上のディレクトリ
STORAGE_DIR_PATH = "/content/drive/MyDrive/drive_ws/marketdata"

# 各種CSVデータを保存するファイルパス
stock_fins_csvfile_path = STORAGE_DIR_PATH + "/stock_fin.csv.gz"
stock_price_csvfile_path = STORAGE_DIR_PATH + "/stock_price.csv.gz"

# J-Quants API から取得するデータの期間
start_dt: datetime = datetime(2017, 1, 1, tzinfo=tz.gettz("Asia/Tokyo"))
# デバッグ中なので短い期間を使う
#start_dt: datetime = datetime(2022, 6, 1, tzinfo=tz.gettz("Asia/Tokyo"))

end_dt: datetime = datetime(2022, 7, 31, tzinfo=tz.gettz("Asia/Tokyo"))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Googleドライブをマウント
from google.colab import drive
drive.mount('/content/drive')

入力ボックスに https://application.jpx-jquants.com/ から取得したJ-Quants APIのリフレッシュトークンを入力してください。

In [3]:
refresh_token = getpass.getpass(prompt="J-Quants Refresh Token:")
# リフレッシュトークンを使用できるか検証します。
test_cli = jquantsapi.Client(refresh_token=refresh_token)
try:
    id_token = test_cli.get_id_token()
    if len(id_token) > 0:
        print("refresh_tokenは正常です。次の手順に進んでください。")
except HTTPError:
    print("refresh_tokenを使用できません。再度値を確認してください。")

J-Quants Refresh Token:··········
refresh_tokenは正常です。次の手順に進んでください。


データ取得期間(start_dt <= 時刻 <= end_dt) の
全銘柄の価格と財務情報データを取得します。
このステップの実行には環境によって数十分ほど時間がかかります。

In [14]:
cli = jquantsapi.Client(refresh_token=refresh_token)
stock_price: pd.DataFrame = cli.get_price_range(start_dt=start_dt, end_dt=end_dt)
stock_fin: pd.DataFrame = cli.get_statements_range(start_dt=start_dt, end_dt=end_dt)

KeyboardInterrupt: ignored

取得したデータをcsv.gz形式で保存します。
stock_priceが巨大なファイル（非圧縮で540MB程度)なので、保存には3分ほどかかることがあります。

In [17]:
stock_price.to_csv(stock_price_csvfile_path, compression="gzip")
stock_fin.to_csv(stock_fins_csvfile_path, compression="gzip")

このnotebookはデータ保存までで終了です。predictor.ipynbに進みましょう。